# Classification Neural Net - NUMPY

In this notebook I will develop a simple classification neural network from scratch using pythons NUMPY, instead of relying on libaries like pytorch.

In [ ]:
import numpy as np
import pandas as pd

First we will define the constants that will be used throughout the notebook.

In [ ]:
weight = 0.5
bias = 0
learning_rate = 0.1
truth = 1 # the value we expect (the actual value that's labeled )

Next we will create a couple functions which will handle the activation, linear neuron output, and then the loss.

In [ ]:
def activation():
    print("sigmoid")

def neuron_output():
    print("output")

def MSE_loss():
    print("MSE_loss")